In [11]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.types import Row

from pyspark.sql import SQLContext

In [41]:
spark      = SparkContext(master='local', appName='DataFrames')
sql_context = SQLContext(spark)

In [47]:
!head -4 /home/cesarppz/Documents/master/code/python/spark/recursos_curso/curso-apache-spark-platzi/files/juegos.csv 

,nombre_juego,annio,temporada,ciudad
1,1896 Verano,1896,Verano,Athina
2,1900 Verano,1900,Verano,Paris
3,1904 Verano,1904,Verano,St. Louis


In [30]:
path = '/home/cesarppz/Documents/master/code/python/spark/recursos_curso/curso-apache-spark-platzi/files/'

### Genrar el esquema

In [42]:
juegoSchema = StructType([
	StructField('juego_id', IntegerType(), False),
    StructField('nombre_juego', StringType(), False),
    StructField('year', IntegerType(), False),
    StructField('temporada', StringType(), False),
    StructField('ciudad', StringType(), False)
])

In [43]:
juegoDF = sql_context.read.schema(juegoSchema).option('header','true').csv(path+'juegos.csv')

Py4JJavaError: An error occurred while calling o126.read.
: java.lang.IllegalStateException: LiveListenerBus is stopped.
	at org.apache.spark.scheduler.LiveListenerBus.addToQueue(LiveListenerBus.scala:98)
	at org.apache.spark.scheduler.LiveListenerBus.addToStatusQueue(LiveListenerBus.scala:81)
	at org.apache.spark.sql.internal.SharedState.<init>(SharedState.scala:115)
	at org.apache.spark.sql.SparkSession.$anonfun$sharedState$1(SparkSession.scala:143)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sharedState$lzycompute(SparkSession.scala:143)
	at org.apache.spark.sql.SparkSession.sharedState(SparkSession.scala:142)
	at org.apache.spark.sql.SparkSession.$anonfun$sessionState$2(SparkSession.scala:162)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sessionState$lzycompute(SparkSession.scala:160)
	at org.apache.spark.sql.SparkSession.sessionState(SparkSession.scala:157)
	at org.apache.spark.sql.DataFrameReader.<init>(

In [49]:
spark.stop()

In [50]:

spark = SparkSession.builder.master("local").appName("test-read").getOrCreate()


In [57]:
sdf = spark.read.schema(juegoSchema).option('header', 'true').csv(path+'juegos.csv')

In [59]:
sdf.show(5)

+--------+------------+----+---------+---------+
|juego_id|nombre_juego|year|temporada|   ciudad|
+--------+------------+----+---------+---------+
|       1| 1896 Verano|1896|   Verano|   Athina|
|       2| 1900 Verano|1900|   Verano|    Paris|
|       3| 1904 Verano|1904|   Verano|St. Louis|
|       4| 1906 Verano|1906|   Verano|   Athina|
|       5| 1908 Verano|1908|   Verano|   London|
+--------+------------+----+---------+---------+
only showing top 5 rows



In [75]:
rddDeportista = spark.sparkContext.textFile(path + 'deportista.csv').map(lambda x : x.split(','))

In [76]:
rddDeportista.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [67]:
def limpiarEncabezado(indice, iterador):
    return iter(list(iterador)[1:])

In [78]:
rddDeportista = rddDeportista.mapPartitionsWithIndex(limpiarEncabezado)

In [79]:
rddDeportista.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [81]:
rddDeportista = rddDeportista.map(lambda x : (
	int(x[0]),
    x[1],
    int(x[2]),
    int(x[3]),
    int(x[4]),
    float(x[5]),
    int(x[6])
))